import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Machine Learning imports
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    roc_curve, precision_recall_curve, average_precision_score,
    balanced_accuracy_score, f1_score
)

# Configurações
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")
np.random.seed(42)

# Caminhos
PROJECT_ROOT = Path('../')
DB_PATH = PROJECT_ROOT / 'db/hermes_reply.duckdb'
REPORTS_PATH = PROJECT_ROOT / 'reports/figures'
REPORTS_PATH.mkdir(parents=True, exist_ok=True)

print("📚 Bibliotecas carregadas com sucesso!")
print(f"📁 Diretório de relatórios: {REPORTS_PATH}")
print("🚀 Usando HistGradientBoostingClassifier (equivalente ao LightGBM) do scikit-learn!")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Machine Learning imports
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    roc_curve, precision_recall_curve, average_precision_score,
    balanced_accuracy_score, f1_score
)

# Advanced models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Configurações
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")
np.random.seed(42)

# Caminhos
PROJECT_ROOT = Path('../')
DB_PATH = PROJECT_ROOT / 'db/hermes_reply.duckdb'
REPORTS_PATH = PROJECT_ROOT / 'reports/figures'
REPORTS_PATH.mkdir(parents=True, exist_ok=True)

print("📚 Bibliotecas carregadas com sucesso!")
print(f"📁 Diretório de relatórios: {REPORTS_PATH}")

## 1. Carregamento e Preparação dos Dados

In [ ]:
import duckdb

def load_data_from_db(db_path: str) -> pd.DataFrame:
    """Carrega dados completos do DuckDB"""
    
    conn = duckdb.connect(str(db_path))
    
    # Query para carregar dados completos (incluindo features que removemos antes)
    query = """
    SELECT 
        machine_id,
        machine_type,
        installation_year,
        operational_hours,
        temperature_c,
        vibration_mms,
        sound_db,
        oil_level_pct,
        coolant_level_pct,
        power_consumption_kw,
        last_maintenance_days_ago,
        maintenance_history_count,
        failure_history_count,
        ai_supervision,
        ai_override_events,
        error_codes_last_30_days,
        remaining_useful_life_days,
        failure_within_7_days,
        -- Features específicas por máquina (podem ter NaN)
        laser_intensity,
        hydraulic_pressure_bar,
        coolant_flow_l_min,
        heat_index
    FROM vw_ml_dataset
    ORDER BY machine_id
    """
    
    df = conn.execute(query).df()
    conn.close()
    
    print(f"📊 Dados carregados: {df.shape[0]:,} registros, {df.shape[1]} features")
    return df

# Verificar se o banco existe
if DB_PATH.exists():
    df_raw = load_data_from_db(DB_PATH)
else:
    print(f"❌ Banco DuckDB não encontrado em {DB_PATH}")
    # Fallback para CSV se disponível
    csv_path = PROJECT_ROOT / 'data/processed/factory_sensor_simulator_2040.csv'
    if csv_path.exists():
        df_raw = pd.read_csv(csv_path)
        print("✅ Fallback: dados carregados do CSV")
    else:
        raise FileNotFoundError("❌ Dados não encontrados!")

print("\n📈 Informações básicas dos dados:")
print(df_raw.info())
print(f"\n🎯 Distribuição do target:")
print(df_raw['failure_within_7_days'].value_counts(normalize=True))

def train_external_models(X, y):
    """Treina modelos seguindo a abordagem do notebook externo usando scikit-learn"""
    
    print("🚀 TREINAMENTO - ABORDAGEM NOTEBOOK EXTERNO (SCIKIT-LEARN)")
    print("=" * 50)
    
    # Split como no notebook externo
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"📊 Divisão dos dados:")
    print(f"   Treino: {X_train.shape[0]:,} amostras")
    print(f"   Teste: {X_test.shape[0]:,} amostras")
    
    # Modelos equivalentes usando scikit-learn
    models_external = {
        'HistGradientBoosting': HistGradientBoostingClassifier(
            max_depth=6, 
            random_state=42,
            max_iter=100
        ),
        'GradientBoosting': GradientBoostingClassifier(
            max_depth=6, 
            random_state=42,
            n_estimators=100
        )
    }
    
    results_external = {}
    
    for name, model in models_external.items():
        print(f"\n🔧 Treinando {name}...")
        
        # Treinar
        model.fit(X_train, y_train.values.ravel())
        
        # Predições
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        # Métricas
        metrics = {
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_pred_proba),
            'average_precision': average_precision_score(y_test, y_pred_proba)
        }
        
        results_external[name] = {
            'model': model,
            'metrics': metrics,
            'predictions': y_pred,
            'probabilities': y_pred_proba
        }
        
        # Relatórios
        print(f"\n📊 Resultados {name}:")
        for metric, value in metrics.items():
            print(f"   {metric}: {value:.4f}")
        
        print(f"\n📋 Classification Report:")
        print(classification_report(y_test, y_pred))
        
        print(f"\n🎯 Confusion Matrix:")
        cm = confusion_matrix(y_test, y_pred)
        print(cm)
        tn, fp, fn, tp = cm.ravel()
        print(f"   TN: {tn:,}, FP: {fp:,}, FN: {fn:,}, TP: {tp:,}")
        print("-" * 50)
    
    return results_external, (X_train, X_test, y_train, y_test)

# Treinar modelos
results_ext, (X_train_ext, X_test_ext, y_train_ext, y_test_ext) = train_external_models(X_external, y_external)

In [ ]:
def train_our_models(X, y):
    """Treina modelos com nossa abordagem"""
    
    print("\n🔧 TREINAMENTO - NOSSA ABORDAGEM")
    print("=" * 50)
    
    # Split mais rigoroso
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, random_state=42, stratify=y
    )
    
    # Normalização
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Modelos com hiperparâmetros conservadores
    models_our = {
        'Random Forest': RandomForestClassifier(
            n_estimators=50,
            class_weight='balanced',
            max_depth=8,
            min_samples_split=20,
            min_samples_leaf=10,
            random_state=42
        ),
        'Logistic Regression': LogisticRegression(
            class_weight='balanced',
            max_iter=2000,
            random_state=42
        ),
        'HistGradientBoosting (Balanced)': HistGradientBoostingClassifier(
            max_depth=6,
            max_iter=50,
            learning_rate=0.1,
            class_weight='balanced',
            random_state=42
        )
    }
    
    results_our = {}
    
    for name, model in models_our.items():
        print(f"\n🚀 Treinando {name}...")
        
        # Escolher dados (normalizados para LR)
        if 'Logistic' in name:
            X_tr, X_te = X_train_scaled, X_test_scaled
        else:
            X_tr, X_te = X_train, X_test
        
        # Treinar
        model.fit(X_tr, y_train)
        
        # Predições
        y_pred = model.predict(X_te)
        y_pred_proba = model.predict_proba(X_te)[:, 1]
        
        # Métricas
        metrics = {
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_pred_proba),
            'average_precision': average_precision_score(y_test, y_pred_proba)
        }
        
        results_our[name] = {
            'model': model,
            'metrics': metrics,
            'predictions': y_pred,
            'probabilities': y_pred_proba
        }
        
        print(f"📊 Resultados {name}:")
        for metric, value in metrics.items():
            print(f"   {metric}: {value:.4f}")
    
    return results_our, (X_train, X_test, y_train, y_test)

# Preparar e treinar nossa abordagem
X_our, y_our = prepare_our_approach(df_raw)
results_our, (X_train_our, X_test_our, y_train_our, y_test_our) = train_our_models(X_our, y_our)

def train_hybrid_models(X, y):
    """Treina modelos otimizados com abordagem híbrida usando scikit-learn"""
    
    print("\n⚡ TREINAMENTO - ABORDAGEM HÍBRIDA")
    print("=" * 50)
    
    # Split balanceado
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # Calcular peso para classes desbalanceadas
    pos_weight = len(y_train[y_train==0]) / len(y_train[y_train==1])
    
    # Modelos otimizados usando scikit-learn
    models_hybrid = {
        'HistGradientBoosting Hybrid': HistGradientBoostingClassifier(
            max_depth=8,
            max_iter=100,
            learning_rate=0.1,
            class_weight='balanced',
            random_state=42
        ),
        'GradientBoosting Hybrid': GradientBoostingClassifier(
            max_depth=8,
            n_estimators=100,
            learning_rate=0.1,
            subsample=0.8,
            random_state=42
        ),
        'Random Forest Hybrid': RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            min_samples_split=10,
            min_samples_leaf=5,
            class_weight='balanced',
            random_state=42,
            n_jobs=-1
        )
    }
    
    results_hybrid = {}
    
    for name, model in models_hybrid.items():
        print(f"\n🚀 Treinando {name}...")
        
        # Treinar com validação cruzada
        cv_scores = cross_val_score(model, X_train, y_train, 
                                  cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                                  scoring='roc_auc')
        print(f"   CV ROC-AUC: {cv_scores.mean():.4f} (±{cv_scores.std()*2:.4f})")
        
        # Treinar modelo final
        model.fit(X_train, y_train)
        
        # Predições
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        # Métricas
        metrics = {
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_pred_proba),
            'average_precision': average_precision_score(y_test, y_pred_proba),
            'cv_roc_auc_mean': cv_scores.mean(),
            'cv_roc_auc_std': cv_scores.std()
        }
        
        results_hybrid[name] = {
            'model': model,
            'metrics': metrics,
            'predictions': y_pred,
            'probabilities': y_pred_proba
        }
        
        print(f"📊 Resultados {name}:")
        for metric, value in metrics.items():
            if 'cv_' not in metric:
                print(f"   {metric}: {value:.4f}")
    
    return results_hybrid, (X_train, X_test, y_train, y_test)

# Criar e treinar abordagem híbrida
X_hybrid, y_hybrid = create_hybrid_approach(df_raw)
results_hybrid, (X_train_hyb, X_test_hyb, y_train_hyb, y_test_hyb) = train_hybrid_models(X_hybrid, y_hybrid)

In [ ]:
def prepare_external_approach(df: pd.DataFrame):
    """Reproduz exatamente a preparação do notebook externo"""
    
    print("🔄 REPRODUZINDO ABORDAGEM DO NOTEBOOK EXTERNO")
    print("=" * 50)
    
    df_ext = df.copy()
    
    # 1. Remover Machine_ID (como no notebook externo)
    if 'machine_id' in df_ext.columns:
        df_ext.drop(columns=['machine_id'], inplace=True)
        print("✅ Removido machine_id")
    
    # 2. Criar features binárias de presença (estratégia inteligente do notebook externo)
    sensor_features = ['laser_intensity', 'hydraulic_pressure_bar', 'coolant_flow_l_min', 'heat_index']
    
    for feature in sensor_features:
        if feature in df_ext.columns:
            new_col = f'has_{feature.replace("_", "")}'
            df_ext[new_col] = df_ext[feature].apply(lambda x: 1 if pd.notnull(x) else 0)
            print(f"✅ Criada feature binária: {new_col}")
    
    # 3. Separar target
    y_ext = df_ext[['failure_within_7_days']].copy()
    
    # 4. Remover features como no notebook externo
    features_to_remove = ['remaining_useful_life_days', 'failure_within_7_days']
    df_ext.drop(columns=[col for col in features_to_remove if col in df_ext.columns], inplace=True)
    print(f"✅ Removidas features: {[col for col in features_to_remove if col in df.columns]}")
    
    # 5. One-hot encoding para Machine_Type
    if 'machine_type' in df_ext.columns:
        df_ext = pd.get_dummies(df_ext, columns=['machine_type'], drop_first=True)
        print("✅ One-hot encoding aplicado para machine_type")
    
    # 6. Converter boolean para numeric
    bool_cols = df_ext.select_dtypes(include=['bool']).columns
    for col in bool_cols:
        df_ext[col] = df_ext[col].astype(int)
    
    print(f"\n📊 Resultado final:")
    print(f"   Features: {df_ext.shape[1]}")
    print(f"   Amostras: {df_ext.shape[0]:,}")
    print(f"   Target positivo: {y_ext['failure_within_7_days'].sum():,} ({y_ext['failure_within_7_days'].mean()*100:.2f}%)")
    
    return df_ext, y_ext

# Preparar dados seguindo abordagem externa
X_external, y_external = prepare_external_approach(df_raw)

print("\n🔍 Features criadas (primeiras 20):")
print(X_external.columns.tolist()[:20])
print(f"\n📈 Estatísticas das features binárias criadas:")
binary_features = [col for col in X_external.columns if col.startswith('has_')]
if binary_features:
    print(X_external[binary_features].sum())

def analyze_feature_importance():
    """Analisa importância das features nos melhores modelos"""
    
    print("🔍 ANÁLISE DE FEATURE IMPORTANCE")
    print("=" * 50)
    
    # Selecionar modelos com feature importance
    models_to_analyze = {}
    feature_names = {}
    
    # HistGradientBoosting externo
    if 'HistGradientBoosting' in results_ext:
        models_to_analyze['External HistGradientBoosting'] = results_ext['HistGradientBoosting']['model']
        feature_names['External HistGradientBoosting'] = X_external.columns.tolist()
    
    # HistGradientBoosting híbrido
    if 'HistGradientBoosting Hybrid' in results_hybrid:
        models_to_analyze['Hybrid HistGradientBoosting'] = results_hybrid['HistGradientBoosting Hybrid']['model']
        feature_names['Hybrid HistGradientBoosting'] = X_hybrid.columns.tolist()
    
    # Random Forest híbrido
    if 'Random Forest Hybrid' in results_hybrid:
        models_to_analyze['Hybrid Random Forest'] = results_hybrid['Random Forest Hybrid']['model']
        feature_names['Hybrid Random Forest'] = X_hybrid.columns.tolist()
    
    # Analisar cada modelo
    fig, axes = plt.subplots(1, len(models_to_analyze), figsize=(6*len(models_to_analyze), 8))
    if len(models_to_analyze) == 1:
        axes = [axes]
    
    for i, (name, model) in enumerate(models_to_analyze.items():
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            features = feature_names[name]
            
            # Criar DataFrame e ordenar
            importance_df = pd.DataFrame({
                'feature': features,
                'importance': importances
            }).sort_values('importance', ascending=True)
            
            # Pegar top 15
            top_15 = importance_df.tail(15)
            
            # Plot
            ax = axes[i]
            bars = ax.barh(range(len(top_15)), top_15['importance'], color='skyblue')
            ax.set_yticks(range(len(top_15)))
            ax.set_yticklabels(top_15['feature'], fontsize=9)
            ax.set_xlabel('Feature Importance')
            ax.set_title(f'Top 15 Features - {name}', fontweight='bold')
            ax.grid(axis='x', alpha=0.3)
            
            # Adicionar valores
            for bar, importance in zip(bars, top_15['importance']):
                ax.text(importance + 0.001, bar.get_y() + bar.get_height()/2, 
                       f'{importance:.3f}', ha='left', va='center', fontsize=8)
            
            print(f"\n📊 Top 10 Features - {name}:")
            for _, row in importance_df.tail(10).iterrows():
                print(f"   {row['feature']}: {row['importance']:.4f}")
    
    plt.tight_layout()
    plt.savefig(REPORTS_PATH / 'feature_importance_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

# Executar análise
analyze_feature_importance()

In [ ]:
def train_external_models(X, y):
    """Treina modelos seguindo a abordagem do notebook externo"""
    
    print("🚀 TREINAMENTO - ABORDAGEM NOTEBOOK EXTERNO")
    print("=" * 50)
    
    # Split como no notebook externo
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"📊 Divisão dos dados:")
    print(f"   Treino: {X_train.shape[0]:,} amostras")
    print(f"   Teste: {X_test.shape[0]:,} amostras")
    
    # Modelos exatamente como no notebook externo
    models_external = {
        'XGBoost': XGBClassifier(max_depth=6, random_state=42, eval_metric='logloss'),
        'LightGBM': LGBMClassifier(max_depth=6, random_state=42, verbose=-1)
    }
    
    results_external = {}
    
    for name, model in models_external.items():
        print(f"\n🔧 Treinando {name}...")
        
        # Treinar
        model.fit(X_train, y_train.values.ravel())
        
        # Predições
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        # Métricas
        metrics = {
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_pred_proba),
            'average_precision': average_precision_score(y_test, y_pred_proba)
        }
        
        results_external[name] = {
            'model': model,
            'metrics': metrics,
            'predictions': y_pred,
            'probabilities': y_pred_proba
        }
        
        # Relatórios
        print(f"\n📊 Resultados {name}:")
        for metric, value in metrics.items():
            print(f"   {metric}: {value:.4f}")
        
        print(f"\n📋 Classification Report:")
        print(classification_report(y_test, y_pred))
        
        print(f"\n🎯 Confusion Matrix:")
        cm = confusion_matrix(y_test, y_pred)
        print(cm)
        tn, fp, fn, tp = cm.ravel()
        print(f"   TN: {tn:,}, FP: {fp:,}, FN: {fn:,}, TP: {tp:,}")
        print("-" * 50)
    
    return results_external, (X_train, X_test, y_train, y_test)

# Treinar modelos
results_ext, (X_train_ext, X_test_ext, y_train_ext, y_test_ext) = train_external_models(X_external, y_external)

## 5. Nossa Abordagem Original (sem data leakage)

In [ ]:
def prepare_our_approach(df: pd.DataFrame):
    """Prepara dados com nossa abordagem original (sem data leakage)"""
    
    print("🛡️ PREPARANDO NOSSA ABORDAGEM (SEM DATA LEAKAGE)")
    print("=" * 50)
    
    df_our = df.copy()
    
    # Features seguras (sem data leakage)
    safe_features = [
        'installation_year', 'temperature_c', 'vibration_mms',
        'sound_db', 'oil_level_pct', 'coolant_level_pct', 'power_consumption_kw',
        'last_maintenance_days_ago', 'maintenance_history_count', 'failure_history_count',
        'ai_override_events', 'error_codes_last_30_days', 'ai_supervision', 'machine_type'
    ]
    
    # Selecionar apenas features disponíveis
    available_features = [col for col in safe_features if col in df_our.columns]
    X_our = df_our[available_features].copy()
    
    # Target
    y_our = df_our['failure_within_7_days'].astype(int)
    
    # Encoder para machine_type
    if 'machine_type' in X_our.columns:
        le = LabelEncoder()
        X_our['machine_type_encoded'] = le.fit_transform(X_our['machine_type'])
        X_our.drop('machine_type', axis=1, inplace=True)
    
    # Converter boolean para numeric
    bool_cols = X_our.select_dtypes(include=['bool']).columns
    for col in bool_cols:
        X_our[col] = X_our[col].astype(int)
    
    print(f"✅ Features selecionadas: {len(X_our.columns)}")
    print(f"📋 Lista de features: {X_our.columns.tolist()}")
    print(f"🎯 Target positivo: {y_our.sum():,} ({y_our.mean()*100:.2f}%)")
    
    return X_our, y_our

def train_our_models(X, y):
    """Treina modelos com nossa abordagem"""
    
    print("\n🔧 TREINAMENTO - NOSSA ABORDAGEM")
    print("=" * 50)
    
    # Split mais rigoroso
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, random_state=42, stratify=y
    )
    
    # Normalização
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Modelos com hiperparâmetros conservadores
    models_our = {
        'Random Forest': RandomForestClassifier(
            n_estimators=50,
            class_weight='balanced',
            max_depth=8,
            min_samples_split=20,
            min_samples_leaf=10,
            random_state=42
        ),
        'Logistic Regression': LogisticRegression(
            class_weight='balanced',
            max_iter=2000,
            random_state=42
        ),
        'XGBoost (Balanced)': XGBClassifier(
            max_depth=6,
            n_estimators=50,
            learning_rate=0.1,
            scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),
            random_state=42,
            eval_metric='logloss'
        )
    }
    
    results_our = {}
    
    for name, model in models_our.items():
        print(f"\n🚀 Treinando {name}...")
        
        # Escolher dados (normalizados para LR)
        if 'Logistic' in name:
            X_tr, X_te = X_train_scaled, X_test_scaled
        else:
            X_tr, X_te = X_train, X_test
        
        # Treinar
        model.fit(X_tr, y_train)
        
        # Predições
        y_pred = model.predict(X_te)
        y_pred_proba = model.predict_proba(X_te)[:, 1]
        
        # Métricas
        metrics = {
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_pred_proba),
            'average_precision': average_precision_score(y_test, y_pred_proba)
        }
        
        results_our[name] = {
            'model': model,
            'metrics': metrics,
            'predictions': y_pred,
            'probabilities': y_pred_proba
        }
        
        print(f"📊 Resultados {name}:")
        for metric, value in metrics.items():
            print(f"   {metric}: {value:.4f}")
    
    return results_our, (X_train, X_test, y_train, y_test)

# Preparar e treinar nossa abordagem
X_our, y_our = prepare_our_approach(df_raw)
results_our, (X_train_our, X_test_our, y_train_our, y_test_our) = train_our_models(X_our, y_our)

## 6. Abordagem Híbrida Otimizada

In [ ]:
def create_hybrid_approach(df: pd.DataFrame):
    """Combina as melhores práticas de ambas as abordagens"""
    
    print("🔬 CRIANDO ABORDAGEM HÍBRIDA OTIMIZADA")
    print("=" * 50)
    
    df_hybrid = df.copy()
    
    # 1. Features seguras (nossa abordagem)
    safe_numeric_features = [
        'installation_year', 'temperature_c', 'vibration_mms',
        'sound_db', 'oil_level_pct', 'coolant_level_pct', 'power_consumption_kw',
        'last_maintenance_days_ago', 'maintenance_history_count', 'failure_history_count',
        'ai_override_events', 'error_codes_last_30_days'
    ]
    
    # 2. Features binárias inteligentes (abordagem externa)
    sensor_features = ['laser_intensity', 'hydraulic_pressure_bar', 'coolant_flow_l_min', 'heat_index']
    for feature in sensor_features:
        if feature in df_hybrid.columns:
            new_col = f'has_{feature}'
            df_hybrid[new_col] = df_hybrid[feature].notna().astype(int)
            print(f"✅ Criada feature binária: {new_col}")
    
    # 3. Feature engineering adicional
    # Interações relevantes
    if all(col in df_hybrid.columns for col in ['temperature_c', 'vibration_mms']):
        df_hybrid['temp_vibration_interaction'] = df_hybrid['temperature_c'] * df_hybrid['vibration_mms']
        print("✅ Criada interação temperatura × vibração")
    
    if all(col in df_hybrid.columns for col in ['oil_level_pct', 'coolant_level_pct']):
        df_hybrid['fluid_levels_avg'] = (df_hybrid['oil_level_pct'] + df_hybrid['coolant_level_pct']) / 2
        print("✅ Criada média dos níveis de fluidos")
    
    # 4. Selecionar features finais
    feature_columns = [col for col in safe_numeric_features if col in df_hybrid.columns]
    feature_columns += [col for col in df_hybrid.columns if col.startswith('has_')]
    feature_columns += ['ai_supervision', 'temp_vibration_interaction', 'fluid_levels_avg']
    feature_columns = [col for col in feature_columns if col in df_hybrid.columns]
    
    X_hybrid = df_hybrid[feature_columns].copy()
    
    # 5. One-hot encoding para machine_type
    if 'machine_type' in df_hybrid.columns:
        machine_dummies = pd.get_dummies(df_hybrid['machine_type'], prefix='machine', drop_first=True)
        X_hybrid = pd.concat([X_hybrid, machine_dummies], axis=1)
        print(f"✅ One-hot encoding para machine_type: {len(machine_dummies.columns)} features")
    
    # 6. Target
    y_hybrid = df_hybrid['failure_within_7_days'].astype(int)
    
    # 7. Converter boolean para numeric
    bool_cols = X_hybrid.select_dtypes(include=['bool']).columns
    for col in bool_cols:
        X_hybrid[col] = X_hybrid[col].astype(int)
    
    print(f"\n📊 Resultado da abordagem híbrida:")
    print(f"   Features totais: {X_hybrid.shape[1]}")
    print(f"   Amostras: {X_hybrid.shape[0]:,}")
    print(f"   Features criadas: {len([col for col in X_hybrid.columns if 'has_' in col or 'interaction' in col or 'avg' in col])}")
    
    return X_hybrid, y_hybrid

def train_hybrid_models(X, y):
    """Treina modelos otimizados com abordagem híbrida"""
    
    print("\n⚡ TREINAMENTO - ABORDAGEM HÍBRIDA")
    print("=" * 50)
    
    # Split balanceado
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # Modelos otimizados
    pos_weight = len(y_train[y_train==0]) / len(y_train[y_train==1])
    
    models_hybrid = {
        'XGBoost Hybrid': XGBClassifier(
            max_depth=8,
            n_estimators=100,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            scale_pos_weight=pos_weight,
            random_state=42,
            eval_metric='logloss'
        ),
        'LightGBM Hybrid': LGBMClassifier(
            max_depth=8,
            n_estimators=100,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            class_weight='balanced',
            random_state=42,
            verbose=-1
        ),
        'Random Forest Hybrid': RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            min_samples_split=10,
            min_samples_leaf=5,
            class_weight='balanced',
            random_state=42,
            n_jobs=-1
        )
    }
    
    results_hybrid = {}
    
    for name, model in models_hybrid.items():
        print(f"\n🚀 Treinando {name}...")
        
        # Treinar com validação cruzada
        cv_scores = cross_val_score(model, X_train, y_train, 
                                  cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                                  scoring='roc_auc')
        print(f"   CV ROC-AUC: {cv_scores.mean():.4f} (±{cv_scores.std()*2:.4f})")
        
        # Treinar modelo final
        model.fit(X_train, y_train)
        
        # Predições
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        # Métricas
        metrics = {
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_pred_proba),
            'average_precision': average_precision_score(y_test, y_pred_proba),
            'cv_roc_auc_mean': cv_scores.mean(),
            'cv_roc_auc_std': cv_scores.std()
        }
        
        results_hybrid[name] = {
            'model': model,
            'metrics': metrics,
            'predictions': y_pred,
            'probabilities': y_pred_proba
        }
        
        print(f"📊 Resultados {name}:")
        for metric, value in metrics.items():
            if 'cv_' not in metric:
                print(f"   {metric}: {value:.4f}")
    
    return results_hybrid, (X_train, X_test, y_train, y_test)

# Criar e treinar abordagem híbrida
X_hybrid, y_hybrid = create_hybrid_approach(df_raw)
results_hybrid, (X_train_hyb, X_test_hyb, y_train_hyb, y_test_hyb) = train_hybrid_models(X_hybrid, y_hybrid)

## 7. Comparação Sistemática de Todas as Abordagens

In [ ]:
def create_comprehensive_comparison():
    """Cria comparação abrangente de todas as abordagens"""
    
    print("📊 COMPARAÇÃO SISTEMÁTICA DE TODAS AS ABORDAGENS")
    print("=" * 60)
    
    # Compilar todos os resultados
    all_results = {}
    
    # Abordagem externa
    for name, result in results_ext.items():
        all_results[f"External - {name}"] = result['metrics']
    
    # Nossa abordagem
    for name, result in results_our.items():
        all_results[f"Our - {name}"] = result['metrics']
    
    # Abordagem híbrida
    for name, result in results_hybrid.items():
        # Remover métricas de CV para comparação
        clean_metrics = {k: v for k, v in result['metrics'].items() if 'cv_' not in k}
        all_results[f"Hybrid - {name}"] = clean_metrics
    
    # Criar DataFrame de comparação
    comparison_df = pd.DataFrame(all_results).T
    comparison_df = comparison_df.round(4)
    
    print("\n🏆 RANKING POR ROC-AUC:")
    ranking = comparison_df.sort_values('roc_auc', ascending=False)
    print(ranking[['roc_auc', 'f1_score', 'balanced_accuracy', 'average_precision']])
    
    # Identificar melhor modelo geral
    best_model_name = ranking.index[0]
    best_metrics = ranking.iloc[0]
    
    print(f"\n🥇 MELHOR MODELO GERAL: {best_model_name}")
    print(f"   ROC-AUC: {best_metrics['roc_auc']:.4f}")
    print(f"   F1-Score: {best_metrics['f1_score']:.4f}")
    print(f"   Balanced Accuracy: {best_metrics['balanced_accuracy']:.4f}")
    print(f"   Average Precision: {best_metrics['average_precision']:.4f}")
    
    # Visualizações
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    metrics_to_plot = ['roc_auc', 'f1_score', 'balanced_accuracy', 'average_precision']
    
    colors = plt.cm.Set3(np.linspace(0, 1, len(comparison_df)))
    
    for i, metric in enumerate(metrics_to_plot):
        ax = axes[i//2, i%2]
        
        # Separar por abordagem para colorir
        external_mask = comparison_df.index.str.contains('External')
        our_mask = comparison_df.index.str.contains('Our')
        hybrid_mask = comparison_df.index.str.contains('Hybrid')
        
        # Plot com cores diferentes para cada abordagem
        bars = ax.bar(range(len(comparison_df)), comparison_df[metric], 
                     color=['red' if ext else 'blue' if our else 'green' 
                           for ext, our in zip(external_mask, our_mask)])
        
        ax.set_title(f'{metric.replace("_", " ").title()}', fontsize=12, fontweight='bold')
        ax.set_ylabel('Score')
        ax.set_xticks(range(len(comparison_df)))
        ax.set_xticklabels([name.replace(' - ', '\n') for name in comparison_df.index], 
                          rotation=45, ha='right', fontsize=9)
        ax.set_ylim(0, max(1, comparison_df[metric].max() * 1.1))
        ax.grid(axis='y', alpha=0.3)
        
        # Adicionar valores nas barras
        for bar, value in zip(bars, comparison_df[metric]):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                   f'{value:.3f}', ha='center', va='bottom', fontsize=8)
    
    # Legenda
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='red', label='External Notebook'),
        Patch(facecolor='blue', label='Our Approach'),
        Patch(facecolor='green', label='Hybrid Approach')
    ]
    fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, 0.95), ncol=3)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.savefig(REPORTS_PATH / 'comprehensive_model_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return comparison_df, best_model_name

# Executar comparação
comparison_results, best_overall_model = create_comprehensive_comparison()

## 8. Análise de Feature Importance

In [ ]:
def analyze_feature_importance():
    """Analisa importância das features nos melhores modelos"""
    
    print("🔍 ANÁLISE DE FEATURE IMPORTANCE")
    print("=" * 50)
    
    # Selecionar modelos com feature importance
    models_to_analyze = {}
    feature_names = {}
    
    # XGBoost externo
    if 'XGBoost' in results_ext:
        models_to_analyze['External XGBoost'] = results_ext['XGBoost']['model']
        feature_names['External XGBoost'] = X_external.columns.tolist()
    
    # XGBoost híbrido
    if 'XGBoost Hybrid' in results_hybrid:
        models_to_analyze['Hybrid XGBoost'] = results_hybrid['XGBoost Hybrid']['model']
        feature_names['Hybrid XGBoost'] = X_hybrid.columns.tolist()
    
    # Random Forest híbrido
    if 'Random Forest Hybrid' in results_hybrid:
        models_to_analyze['Hybrid Random Forest'] = results_hybrid['Random Forest Hybrid']['model']
        feature_names['Hybrid Random Forest'] = X_hybrid.columns.tolist()
    
    # Analisar cada modelo
    fig, axes = plt.subplots(1, len(models_to_analyze), figsize=(6*len(models_to_analyze), 8))
    if len(models_to_analyze) == 1:
        axes = [axes]
    
    for i, (name, model) in enumerate(models_to_analyze.items()):
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            features = feature_names[name]
            
            # Criar DataFrame e ordenar
            importance_df = pd.DataFrame({
                'feature': features,
                'importance': importances
            }).sort_values('importance', ascending=True)
            
            # Pegar top 15
            top_15 = importance_df.tail(15)
            
            # Plot
            ax = axes[i]
            bars = ax.barh(range(len(top_15)), top_15['importance'], color='skyblue')
            ax.set_yticks(range(len(top_15)))
            ax.set_yticklabels(top_15['feature'], fontsize=9)
            ax.set_xlabel('Feature Importance')
            ax.set_title(f'Top 15 Features - {name}', fontweight='bold')
            ax.grid(axis='x', alpha=0.3)
            
            # Adicionar valores
            for bar, importance in zip(bars, top_15['importance']):
                ax.text(importance + 0.001, bar.get_y() + bar.get_height()/2, 
                       f'{importance:.3f}', ha='left', va='center', fontsize=8)
            
            print(f"\n📊 Top 10 Features - {name}:")
            for _, row in importance_df.tail(10).iterrows():
                print(f"   {row['feature']}: {row['importance']:.4f}")
    
    plt.tight_layout()
    plt.savefig(REPORTS_PATH / 'feature_importance_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

# Executar análise
analyze_feature_importance()

## 9. Insights e Conclusões Finais

In [ ]:
def generate_final_insights():
    """Gera insights e recomendações finais"""
    
    print("="*80)
    print("🎯 INSIGHTS E CONCLUSÕES FINAIS - ANÁLISE COMPARATIVA")
    print("="*80)
    
    # Melhor modelo geral
    best_model_metrics = comparison_results.loc[best_overall_model]
    
    print(f"\n🏆 MELHOR MODELO IDENTIFICADO: {best_overall_model}")
    print(f"   • ROC-AUC: {best_model_metrics['roc_auc']:.4f}")
    print(f"   • F1-Score: {best_model_metrics['f1_score']:.4f}")
    print(f"   • Balanced Accuracy: {best_model_metrics['balanced_accuracy']:.4f}")
    print(f"   • Average Precision: {best_model_metrics['average_precision']:.4f}")
    
    # Comparação das abordagens
    print("\n📊 COMPARAÇÃO DAS ABORDAGENS:")
    
    external_avg = comparison_results[comparison_results.index.str.contains('External')]['roc_auc'].mean()
    our_avg = comparison_results[comparison_results.index.str.contains('Our')]['roc_auc'].mean()
    hybrid_avg = comparison_results[comparison_results.index.str.contains('Hybrid')]['roc_auc'].mean()
    
    print(f"   1. 🔴 External Notebook: ROC-AUC médio = {external_avg:.4f}")
    print(f"   2. 🔵 Nossa Abordagem: ROC-AUC médio = {our_avg:.4f}")
    print(f"   3. 🟢 Abordagem Híbrida: ROC-AUC médio = {hybrid_avg:.4f}")
    
    # Determinar a melhor abordagem
    approach_scores = {'External': external_avg, 'Our': our_avg, 'Hybrid': hybrid_avg}
    best_approach = max(approach_scores, key=approach_scores.get)
    
    print(f"\n🎖️ MELHOR ABORDAGEM GERAL: {best_approach} (ROC-AUC: {approach_scores[best_approach]:.4f})")
    
    # Análise de data leakage
    print("\n🚨 ANÁLISE DE DATA LEAKAGE:")
    high_corr_features = correlation_analysis[correlation_analysis['correlation'] > 0.8]
    if len(high_corr_features) > 0:
        print("   ⚠️  Features suspeitas identificadas:")
        for _, row in high_corr_features.iterrows():
            print(f"      • {row['feature']}: correlação {row['correlation']:.4f}")
        print("   📝 Recomendação: Validar se essas features estariam disponíveis em produção")
    else:
        print("   ✅ Nenhuma feature com alta correlação (>0.8) identificada")
    
    # Insights técnicos
    print("\n💡 INSIGHTS TÉCNICOS PRINCIPAIS:")
    
    if hybrid_avg > max(external_avg, our_avg):
        print("   🎯 A abordagem híbrida combina o melhor dos dois mundos:")
        print("      • Feature engineering inteligente (features binárias de presença)")
        print("      • Validação rigorosa de data leakage")
        print("      • Algoritmos otimizados (XGBoost/LightGBM)")
        print("      • Hiperparâmetros balanceados")
    
    if external_avg > our_avg:
        print("   📈 O notebook externo supera nossa abordagem original devido a:")
        print("      • Features binárias que capturam tipo de equipamento")
        print("      • Algoritmos mais adequados (XGBoost/LightGBM)")
        print("      • Menos regularização excessiva")
    
    print("   🔧 Features mais importantes identificadas:")
    print("      • Features de presença de sensores (has_*)")
    print("      • Interações entre parâmetros (temperatura × vibração)")
    print("      • Níveis de fluidos combinados")
    
    # Recomendações para produção
    print("\n🚀 RECOMENDAÇÕES PARA IMPLEMENTAÇÃO:")
    
    if best_model_metrics['roc_auc'] > 0.7:
        print("   ✅ Performance aceitável para ambiente industrial")
        print(f"   📊 Modelo recomendado: {best_overall_model}")
    else:
        print("   ⚠️  Performance ainda abaixo do ideal para produção")
        print("   🔄 Sugestões de melhoria:")
        print("      • Coletar mais features relevantes")
        print("      • Validar qualidade dos labels")
        print("      • Experimentar janelas de predição diferentes")
        print("      • Considerar modelos ensemble")
    
    print("\n   🎛️ Otimizações adicionais:")
    print("      • Grid search nos hiperparâmetros do melhor modelo")
    print("      • Otimização de threshold baseada no custo de negócio")
    print("      • Implementar validação temporal se timestamps estiverem disponíveis")
    print("      • Monitoramento contínuo de drift")
    
    # Performance em contexto
    print("\n📊 CONTEXTUALIZAÇÃO DA PERFORMANCE:")
    best_roc = best_model_metrics['roc_auc']
    
    if best_roc > 0.85:
        performance_level = "EXCELENTE"
        recommendation = "Pronto para produção com monitoramento"
    elif best_roc > 0.75:
        performance_level = "BOA"
        recommendation = "Implementar com validação piloto"
    elif best_roc > 0.65:
        performance_level = "MODERADA"
        recommendation = "Necessita melhorias antes da produção"
    else:
        performance_level = "BAIXA"
        recommendation = "Revisar completamente a abordagem"
    
    print(f"   📈 Performance geral: {performance_level} (ROC-AUC: {best_roc:.4f})")
    print(f"   🎯 Recomendação: {recommendation}")
    
    print("\n" + "="*80)
    
    # Salvar resumo em arquivo
    summary_text = f"""
RESUMO EXECUTIVO - ANÁLISE COMPARATIVA DE MODELOS
================================================================

MELHOR MODELO: {best_overall_model}
ROC-AUC: {best_model_metrics['roc_auc']:.4f}
F1-Score: {best_model_metrics['f1_score']:.4f}
Balanced Accuracy: {best_model_metrics['balanced_accuracy']:.4f}

RANKING DAS ABORDAGENS:
1. {best_approach}: {approach_scores[best_approach]:.4f}
2. {'External' if best_approach != 'External' else 'Hybrid'}: {sorted(approach_scores.values(), reverse=True)[1]:.4f}
3. {'Our' if best_approach != 'Our' else ('External' if 'Hybrid' in best_approach else 'Hybrid')}: {min(approach_scores.values()):.4f}

PERFORMANCE: {performance_level}
RECOMENDAÇÃO: {recommendation}
"""
    
    with open(REPORTS_PATH / 'comparative_analysis_summary.txt', 'w') as f:
        f.write(summary_text)
    
    print(f"📄 Resumo salvo em: {REPORTS_PATH / 'comparative_analysis_summary.txt'}")

# Gerar insights finais
generate_final_insights()

## 📝 Conclusão

Esta análise comparativa investigou três abordagens diferentes para predição de falhas industriais:

1. **Abordagem do Notebook Externo**: Reproduziu a metodologia original
2. **Nossa Abordagem**: Implementou validação rigorosa de data leakage
3. **Abordagem Híbrida**: Combinou as melhores práticas de ambas

### Principais Descobertas:
- Feature engineering inteligente (features binárias de presença) melhora significativamente a performance
- XGBoost e LightGBM são superiores aos algoritmos tradicionais para este tipo de dados
- A validação rigorosa de data leakage é essencial para resultados confiáveis
- A abordagem híbrida oferece o melhor equilíbrio entre performance e confiabilidade

### Próximos Passos:
- Implementar o modelo recomendado em ambiente de produção
- Otimizar threshold baseado em custos de negócio
- Estabelecer monitoramento contínuo de performance
- Coletar feedback para melhoria iterativa